In [ ]:
import json
import pprint
import requests
from requests.auth import HTTPBasicAuth
from shapely.geometry import Polygon, shape
import geopandas as gpd

In [ ]:
# Enter planetscope api key here, and adjust other scene query parameters
API_KEY = ''
MINIMUM_AREA_COVERAGE = 0.49
MAXIMUM_CLOUD_COVER = 0.05
ORDER_BATCH_SIZE = 500

In [ ]:
def get_aoi(shp_path):
    gdf = gpd.read_file(shp_path)
    gdf_re = gdf.to_crs(epsg=4326)
    fcollec = json.loads(gdf_re.geometry.to_json())
    return fcollec["features"][0]["geometry"]

def get_polygon(aoi_poly):
    return Polygon(shape(aoi_poly))

In [ ]:
class Scenes:
    def __init__(self, shp_path):
        self.aoi = get_aoi(shp_path)
        self.aoi_polygon = get_polygon(self.aoi)
        self.auth = HTTPBasicAuth(API_KEY, '')

    def search_planet_api(self, cloud_cover=MAXIMUM_CLOUD_COVER):
        geometry_filter = {
            "type": "GeometryFilter",
            "field_name": "geometry",
            "config": self.aoi
            }
        cloud_cover_filter = {
            "type": "RangeFilter",
            "field_name": "cloud_cover",
            "config": {"lte": cloud_cover}
        }
        combined_filter = {
            "type": "AndFilter",
            "config": [geometry_filter, cloud_cover_filter]
            }
        search_request = {
            "item_types": ["PSScene"], 
            "filter": combined_filter
            }
        session = requests.Session()
        search = session.post(
            'https://api.planet.com/data/v1/quick-search',
            auth=self.auth,
            json=search_request)
        search_response = search.json()
        last_response = search_response
        while ('_next' in last_response['_links'] and last_response['_links']['_next'] is not None):
            response = requests.get(last_response['_links']['_next'], auth=self.auth)
            last_response = response.json()
            if len(last_response['features']) == 0:
                break
            search_response['features'] += last_response['features']

        return search_response

    def trim_order(self, search_response, min_coverage=MINIMUM_AREA_COVERAGE):
        total_area = 0
        nb_images = 0
        item_ids = []
        count = 0
        for search_item in search_response['features']:
            geometry_polygon = get_polygon(search_item['geometry'])
            intersection_polygon = self.aoi_polygon.intersection(geometry_polygon)
            ratio = intersection_polygon.area / self.aoi_polygon.area
            count +=1
            if 'assets.ortho_analytic_4b_sr:download' not in search_item['_permissions']:
                #print(search_item['id'])
                continue
            if (ratio < min_coverage):
                continue
            if (not search_item['properties']['ground_control']):
                continue
            item_ids.append(search_item['id'])
            total_area += intersection_polygon.area
            nb_images += 1

        print ('Polygon area:', self.aoi_polygon.area / 1000000, 'km²')
        print ('Total area downloaded:', total_area / 1000000, 'km²')
        print ('Number of images:', nb_images)
        print ('Total images:', count)
        return item_ids

    def place_order(self, item_ids, batch_size=ORDER_BATCH_SIZE, zip_prefix='order_i_'):
        order_url = "https://api.planet.com/compute/ops/orders/v2"
        orders_responses = []
        for i in range(0, len(item_ids), batch_size):
            batch_item_ids = item_ids[i:i + batch_size]
            print('Len of batch: {}'.format(len(batch_item_ids)))
            order_request = {
                "name": "shp_clipped_order", #+ str(i) + "_order",
                "products": [
                    {
                        "item_ids": batch_item_ids,
                        "item_type": "PSScene",
                        "product_bundle": 'analytic_sr_udm2'
                    }
                ],
                "tools": [
                    {
                        "clip": {
                            "aoi": self.aoi
                        }
                    },
                    {
                        "harmonize": {
                            "target_sensor": "Sentinel-2"
                        }
                    },
                    {
                        "reproject": {
                            "projection": "EPSG:32736",
                            "kernel": "cubic"
                        }
                    },
                ],
                "delivery": {
                    "single_archive": True,
                    "archive_type": "zip",
                    "archive_filename": zip_prefix + str(i) + ".zip"
                }
            }

            order = requests.post(order_url, auth=self.auth, json=order_request)
            order_json = order.json()
            print(order_json)
            orders_responses.append(order_json['id'])
        return orders_responses

    def download_order(self, order_id):
        session_ord = requests.Session()
        order_url = f"https://api.planet.com/compute/ops/orders/v2/{order_id}"
        response = session_ord.get(order_url, auth=self.auth)
        order_details = response.json()

        if order_details['state'] == 'success':
            results = order_details['_links']['results']
            for result in results:
                download_url = result['location']
                name = result['name']
                filename = name.split('/')[-1]
                response = session_ord.get(download_url, stream=True)
                with open(filename, 'wb') as file:
                    for chunk in response.iter_content(chunk_size=1024*1024):
                        if chunk:
                            file.write(chunk)
            print('Downloaded clipped images.')
            return True
        else:
            print('Order {} is not yet ready. Retrying in a few moments...'.format(order_id))
            print(order_details)
            return False


In [ ]:
# Enter path of vector file (.shp, .geojson, etc) which contains area of interest to download
aoi_filepath = ''
api_handler = Scenes(aoi_filepath)

In [ ]:
search_info = api_handler.search_planet_api(cloud_cover=0.05)
print('Num scenes: ', len(search_info['features']))

In [ ]:
# intermediate file only for info
with open('search_info.json', 'w') as file:
    pprint.pprint(search_info, stream=file)

In [ ]:
scene_ids = api_handler.trim_order(search_info, min_coverage=0.49)
print('After trimming, num scenes: ', len(scene_ids))

In [ ]:
order_ids = api_handler.place_order(scene_ids, batch_size=400)
print('Num orders: ', len(order_ids))
print(order_ids)

In [ ]:
# replace with path of the file which will store the order_ids (needed to download order in next step)
with open('order_ids.txt', 'w') as file:
    for item in order_ids:
        file.write(f'{item}\n')